In [1]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
API_ENDPOINT = "http://127.0.0.1:5050/predict"
IMAGES_DIR = 'save/images'
OUTPUT_CSV = 'csv_files'
DF_SPLITS_ROW = 500

In [3]:
df = pd.read_csv('../data/raw/Nov_01st_to_07th.csv')
df.head()

,Date,Time,Longitude,Latitude,Image,OutletCode,OutletName,OutletCategory1,OutletCategory2,OutletCategory3,OutletCategory4,UserName,UserRole,Category,NodeName
0,1/11/2024 0:00,9:58:00,80.25455,7.23361,https://surge.lk/Myfiles/RetailerImages/138228...,IC024745,ASHOKA STO.,ICE CREAM_OUTLET HIERARCHY,GENERAL TRADE MAIN (N),GENERAL TRADE MAIN (N) / GROCERIES (N),GENERAL TRADE MAIN (N) / GROCERIES (N) / GRADE...,H.Sasindu Lakshan 67103,CSR,1M Visibility,H.Sasindu Lakshan 67103
1,1/11/2024 0:00,10:13:00,80.24391,7.23137,https://surge.lk/Myfiles/RetailerImages/138569...,IC046840,Aluth Kade,ICE CREAM_OUTLET HIERARCHY,GENERAL TRADE MAIN (N),GENERAL TRADE MAIN (N) / GROCERIES (N),GENERAL TRADE MAIN (N) / GROCERIES (N) / GRADE...,H.Sasindu Lakshan 67103,CSR,1M Visibility,H.Sasindu Lakshan 67103
2,1/11/2024 0:00,10:13:00,80.24389,7.23137,https://surge.lk/Myfiles/RetailerImages/138569...,IC046840,Aluth Kade,ICE CREAM_OUTLET HIERARCHY,GENERAL TRADE MAIN (N),GENERAL TRADE MAIN (N) / GROCERIES (N),GENERAL TRADE MAIN (N) / GROCERIES (N) / GRADE...,H.Sasindu Lakshan 67103,CSR,5M Visibility,H.Sasindu Lakshan 67103
3,1/11/2024 0:00,10:59:00,80.24363,7.23129,https://surge.lk/Myfiles/RetailerImages/306129...,IC158725,Online super,ICE CREAM_OUTLET HIERARCHY,GENERAL TRADE MAIN (N),GENERAL TRADE MAIN (N) / GROCERIES (N),GENERAL TRADE MAIN (N) / GROCERIES (N) / GRADE...,H.Sasindu Lakshan 67103,CSR,1M Visibility,H.Sasindu Lakshan 67103
4,1/11/2024 0:00,11:19:00,80.23432,7.23823,https://surge.lk/Myfiles/RetailerImages/240896...,IC140438,Mihiri Stores,ICE CREAM_OUTLET HIERARCHY,GENERAL TRADE MAIN (N),GENERAL TRADE MAIN (N) / GROCERIES (N),GENERAL TRADE MAIN (N) / GROCERIES (N) / GRADE...,H.Sasindu Lakshan 67103,CSR,1M Visibility,H.Sasindu Lakshan 67103


In [4]:
if not os.path.exists(IMAGES_DIR):
    os.makedirs(IMAGES_DIR)

if not os.path.exists(OUTPUT_CSV):
            os.makedirs(OUTPUT_CSV)

In [5]:
def download_image(url, filename):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
        else:
            print(f"Error downloading image from {url}")
            return False
    except Exception as e:
        print(f"Exception occurred while downloading image from {url}: {e}")
        return False
    return True

In [6]:
def get_prediction(image_path):
    try:
        with open(image_path, 'rb') as f:
            files = {'image': f}
            response = requests.post(API_ENDPOINT, files=files)
            if response.status_code == 200:
                return response.json().get('board_types')
            else:
                print(f"Error with API response: {response.status_code}")
                return 'Error'
    except FileNotFoundError:
        print(f"File not found for prediction: {image_path}")
        return 'File not found'
    except Exception as e:
        print(f"Exception occurred while getting prediction: {e}")
        return 'Error'

In [7]:
def process_image(row):
    image_url = row['Image']
    image_name = os.path.basename(image_url)
    image_path = os.path.join(IMAGES_DIR, image_name)

    if download_image(image_url, image_path):
        prediction = get_prediction(image_path)
        os.remove(image_path)
    else:
        prediction = 'link not working'
    
    return prediction

In [8]:
def predictions_df(df):
    predictions = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        image_url = row['Image']
        image_name = os.path.basename(image_url)
        image_path = os.path.join(IMAGES_DIR, image_name)

       
        if not download_image(image_url, image_path):
            predictions.append('Link not working')
            continue
        
       
        prediction = get_prediction(image_path)
        predictions.append(prediction)
        
        
        try:
            os.remove(image_path)
        except Exception as e:
            print(f"Exception occurred while removing image file: {e}")

    return predictions

In [9]:
def remove_brackets(value):
    if isinstance(value, list):
        return ', '.join(f'{item[0]} {item[1]:.2f}' for item in value) if value else 'none'
    return value

In [10]:
def process_df(df):
    for i in range(0,len(df),DF_SPLITS_ROW):
        proc_df = df.iloc[i:i+DF_SPLITS_ROW]

        preds = predictions_df(proc_df)

        proc_df['Prediction'] = preds
        proc_df['Prediction'] = proc_df['Prediction'].apply(remove_brackets)


        csv_name = f'{OUTPUT_CSV}/output_{i}.csv'
        proc_df.to_csv(csv_name, index=False)

In [11]:
process_df(df)

  0%|          | 0/500 [00:00<?, ?it/s]

  7%|▋         | 35/500 [04:00<40:10,  5.18s/it]  

Error downloading image from https://surge.lk/Myfiles/RetailerImages/1359342513scaled_fedbb070-98e5-46dc-a0c1-b5b1e2027e9f195223000069533293.jpg


  7%|▋         | 36/500 [04:02<32:05,  4.15s/it]

Error downloading image from https://surge.lk/Myfiles/RetailerImages/1359342513scaled_5893e2e5-0020-4bf4-bf7c-8d2185c64cff9185153058143696034.jpg


...

 78%|███████▊  | 166/214 [13:52<04:59,  6.23s/it]

Error downloading image from https://surge.lk/Myfiles/RetailerImages/1359262513scaled_578b9591-bd55-4161-a177-057e48179dfa212829129348238398.jpg


 82%|████████▏ | 176/214 [14:52<03:06,  4.90s/it]

Error downloading image from https://surge.lk/Myfiles/RetailerImages/1358852513scaled_7e739ce7-c434-4d66-920b-234113d1ec2c2910969236623136801.jpg


100%|██████████| 214/214 [19:26<00:00,  5.45s/it]
C:\Users\PC\AppData\Local\Temp\ipykernel_8092\3679421408.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_df['Prediction'] = preds
C:\Users\PC\AppData\Local\Temp\ipykernel_8092\3679421408.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_df['Prediction'] = proc_df['Prediction'].apply(remove_brackets)
